In [16]:
import csv
import pandas as pd
from sklearn import preprocessing

In [9]:
reviews_df = pd.read_csv('reviews_categorical.csv')
reviews_df.columns = ["userID", "productID", "vote", "rating", "verified"]
reviews_df = reviews_df.sample(n=100000, replace=False, random_state=3)

In [34]:
reviews_df.head(3)

,userID,productID,vote,rating,verified
411783,AP4LPGVNZAZ8E,B00ZQB28XK,28,1,0
447376,A3TEDIF9P1FUSW,B00002ST36,449,1,1
142,A2F4Q24VGS5U4H,7293000936,449,1,1


In [37]:
len(set(reviews_df["userID"]))

43522

## META

In [38]:
meta_df = pd.read_json('meta.json')

In [40]:
meta_df = meta_df.T
meta_df.head(5)

,category,tech1,description,fit,title,also_buy,tech2,brand,feature,rank,also_view,main_cat,similar_item,date,price,asin,imageURL,imageURLHighRes,details
0,"[Video Games, PC, Games]",,[],,Reversi Sensory Challenger,[],,Fidelity Electronics,[],"[>#2,623,937 in Toys &amp; Games (See Top 100 ...",[],Toys &amp; Games,,,,0042000742,[https://images-na.ssl-images-amazon.com/image...,[https://images-na.ssl-images-amazon.com/image...,NaN
1,"[Video Games, Xbox 360, Games, </span></span><...",,[Brand new sealed!],,Medal of Honor: Warfighter - Includes Battlefi...,[B00PADROYW],,by\n \n EA Games,[],"[>#67,231 in Video Games (See Top 100 in Video...","[B0050SY5BM, B072NQJCW5, B000TI836G, B002SRSQ7...",Video Games,,,"\n\t\t\t\t\t\t\t\t\t\t\t\t<span class=""vertica...",0078764343,[https://images-na.ssl-images-amazon.com/image...,[https://images-na.ssl-images-amazon.com/image...,NaN
2,"[Video Games, Retro Gaming & Microconsoles, Su...",,[],,street fighter 2 II turbo super nintendo snes ...,[],,Nintendo,[],"[>#134,433 in Video Games (See Top 100 in Vide...",[],Video Games,,,$0.72,0276425316,[],[],NaN
3,"[Video Games, Xbox 360, Accessories, Controlle...",,[MAS's Pro Xbox 360 Stick (Perfect 360 Stick) ...,,Xbox 360 MAS STICK,[],,by\n \n MAS SYSTEMS,[Original PCB used from Xbox 360 Control Pad (...,"[>#105,263 in Video Games (See Top 100 in Vide...",[],Video Games,,,,0324411812,[https://images-na.ssl-images-amazon.com/image...,[https://images-na.ssl-images-amazon.com/image...,NaN
4,"[Video Games, PC, Games, </span></span></span>...",,"[Phonics Alive! 3, The Speller teaches student...",,Phonics Alive! 3: The Speller,[],,by\n \n Advanced Software Pty. Ltd.,"[Grades 2-12, Spelling Program, Teaches Spelli...","[>#92,397 in Video Games (See Top 100 in Video...",[B000BCZ7U0],Video Games,,,,0439335310,[https://images-na.ssl-images-amazon.com/image...,[https://images-na.ssl-images-amazon.com/image...,NaN


In [44]:
# set( meta_df["date"] )

In [45]:
del meta_df["tech1"]
del meta_df["fit"] 
del meta_df["also_buy"] 
del meta_df["tech2"] 
del meta_df["rank"] 
del meta_df["also_view"] 
del meta_df["main_cat"] 
del meta_df["similar_item"] 
del meta_df["imageURL"]
del meta_df["imageURLHighRes"]
del meta_df["details"]

In [11]:
# meta_df = pd.read_csv('meta_filtered.csv')

In [47]:
meta_df.head(3)

,category,description,title,brand,feature,date,price,asin
0,"[Video Games, PC, Games]",[],Reversi Sensory Challenger,Fidelity Electronics,[],,,0042000742
1,"[Video Games, Xbox 360, Games, </span></span><...",[Brand new sealed!],Medal of Honor: Warfighter - Includes Battlefi...,by\n \n EA Games,[],,"\n\t\t\t\t\t\t\t\t\t\t\t\t<span class=""vertica...",0078764343
2,"[Video Games, Retro Gaming & Microconsoles, Su...",[],street fighter 2 II turbo super nintendo snes ...,Nintendo,[],,$0.72,0276425316


In [48]:
meta_df['price'] = meta_df['price'].astype(str)
meta_df['brand'] = meta_df['brand'].astype(str)
meta_df['category'] = meta_df['category'].astype(str)
meta_df['description'] = meta_df['description'].astype(str)
meta_df['date'] = meta_df['date'].astype(str)
meta_df['title'] = meta_df['title'].astype(str)
meta_df['feature'] = meta_df['feature'].astype(str)

In [49]:
def filter_prices(x):
    if x.startswith('$'):
        return x
    else:
        return ''
    
meta_df["price"] = meta_df["price"].apply(lambda x: filter_prices(x))

In [50]:
le = preprocessing.LabelEncoder()
le.fit(meta_df["price"])
new_price = le.transform(meta_df["price"])
meta_df["price"] = new_price

In [55]:
le = preprocessing.LabelEncoder()
le.fit(meta_df["date"])
new_date = le.transform(meta_df["date"])
meta_df["date"] = new_date

In [51]:
meta_df['description'] = meta_df['description'].str.strip('[]').str.split(',')
meta_df['feature'] = meta_df['feature'].str.strip('[]').str.split(',')
meta_df['category'] = meta_df['category'].str.strip('[]').str.split(',')

In [58]:
def get_cat2(x):
    if len(x) >= 2:
        return x[1].replace("'", '')
    else:
        return ''
    
def get_cat3(x):
    if len(x) >= 3:
        return x[2].replace("'", '')
    else:
        return ''
    
def get_description(x):
    if len(x) >= 1:
        return x[0].replace("'", '')
    else:
        return ''

def get_feature(x):
    if len(x) >= 1:
        return x[0].replace("'", '')
    else:
        return ''
    
# meta_df["category2"] = meta_df["category"].apply(lambda x: get_cat2(x))
# meta_df["category3"] = meta_df["category"].apply(lambda x: get_cat3(x))

In [59]:
meta_df["description"] = meta_df["description"].apply(lambda x: get_description(x))
meta_df["feature"] = meta_df["feature"].apply(lambda x: get_feature(x))

In [53]:
del meta_df["category"]

In [64]:
meta_df = meta_df.rename(columns={"asin": "productID"})

In [65]:
meta_df.head(3)

,description,title,brand,feature,date,price,productID,category2,category3
0,,Reversi Sensory Challenger,Fidelity Electronics,,0,0,0042000742,PC,Games
1,Brand new sealed!,Medal of Honor: Warfighter - Includes Battlefi...,by\n \n EA Games,,0,0,0078764343,Xbox 360,Games
2,,street fighter 2 II turbo super nintendo snes ...,Nintendo,,0,16,0276425316,Retro Gaming & Microconsoles,Super Nintendo


In [67]:
# result = pd.concat([meta_df, reviews_df], axis=1, ignore_index=True, sort=False)
result = pd.merge(meta_df, reviews_df, on='productID')
result.head(10)

,description,title,brand,feature,date,price,productID,category2,category3,userID,vote,rating,verified
0,ANNO 2070BRAND NEW - IN STOCKDVD Rom Software ...,Anno 2070,Ubisoft,A new era: while adhering to the fundamentals ...,0,3471,0700026657,PC,Games,A1EO9BFUHTGWKZ,449,1,1
1,ANNO 2070BRAND NEW - IN STOCKDVD Rom Software ...,Anno 2070,Ubisoft,A new era: while adhering to the fundamentals ...,0,3471,0700026657,PC,Games,A1HP7NVNPFMA4N,449,1,1
2,ANNO 2070BRAND NEW - IN STOCKDVD Rom Software ...,Anno 2070,Ubisoft,A new era: while adhering to the fundamentals ...,0,3471,0700026657,PC,Games,AV969NA4CBP10,449,1,1
3,ANNO 2070BRAND NEW - IN STOCKDVD Rom Software ...,Anno 2070,Ubisoft,A new era: while adhering to the fundamentals ...,0,3471,0700026657,PC,Games,A2RF5B5H74JLPE,449,1,1
4,Hitman: Absolution follows Agent 47,HITMAN ABSOLUTION (PC),Square Enix,PC-DVD Import,0,0,0700026398,PC,Games,A1NQ759X8WPIVV,449,0,1
5,Hitman: Absolution follows Agent 47,HITMAN ABSOLUTION (PC),Square Enix,PC-DVD Import,0,0,0700026398,PC,Games,A2RGUDIF7VB7JZ,449,1,1
6,Dirt 3 is a popular rally racing game for Play...,Dirt 3,Dirt 3,DiRT 3 delivers mud,0,0,0700099867,PC,Games,A361M14PU2GUEG,127,1,1
7,Dirt 3 is a popular rally racing game for Play...,Dirt 3,Dirt 3,DiRT 3 delivers mud,0,0,0700099867,PC,Games,A20DRRKAN5Z9Q,449,1,0
8,Dirt 3 is a popular rally racing game for Play...,Dirt 3,Dirt 3,DiRT 3 delivers mud,0,0,0700099867,PC,Games,A1BHRNLW2L8KLD,449,0,1
9,Dirt 3 is a popular rally racing game for Play...,Dirt 3,Dirt 3,DiRT 3 delivers mud,0,0,0700099867,PC,Games,A1QJJU33VNC4S7,449,1,0


In [68]:
len(result)

114394

In [70]:
result_df = result.sample(n=100000, replace=False, random_state=63)
result_df.to_csv("100_real_data.csv", index=False)